In [78]:
# Check if project packages are installed
! pip show selenium webdriver-manager pandas beautifulsoup4 py2neo python-dotenv

Name: selenium
Version: 3.141.0
Summary: Python bindings for Selenium
Home-page: https://github.com/SeleniumHQ/selenium/
Author: UNKNOWN
Author-email: UNKNOWN
License: Apache 2.0
Location: c:\users\joel\anaconda3\lib\site-packages
Requires: urllib3
Required-by: 
---
Name: webdriver-manager
Version: 3.2.1
Summary: Library provides the way to automatically manage drivers for different browsers
Home-page: https://github.com/SergeyPirogov/webdriver_manager
Author: Sergey Pirogov
Author-email: automationremarks@gmail.com
License: UNKNOWN
Location: c:\users\joel\anaconda3\lib\site-packages
Requires: crayons, configparser, requests
Required-by: 
---
Name: pandas
Version: 0.24.1
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: http://pandas.pydata.org
Author: None
Author-email: None
License: BSD
Location: c:\users\joel\anaconda3\lib\site-packages
Requires: numpy, pytz, python-dateutil
Required-by: statsmodels, seaborn
---
Name: beautifulsoup4
Version:

In [79]:
# install packages if not already installed
! pip install selenium webdriver-manager pandas beautifulsoup4 py2neo python-dotenv

In [86]:
# Import packages
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager 
from bs4 import BeautifulSoup
import pandas as pd
import re
import os
from py2neo import Graph, Node, Relationship, NodeMatcher, RelationshipMatcher

In [28]:
#URL constants
GOV_UK_HOME_URL = "https://www.gov.uk"
DOCUMENTS_URL = "https://www.gov.uk/guidance/immigration-rules"
IMMIGRATION_RULES_INDEX_URL = "https://www.gov.uk/guidance/immigration-rules/immigration-rules-index"
POINT_BASED_SYSTEM_URL = "https://www.gov.uk/guidance/immigration-rules/immigration-rules-part-6a-the-points-based-system"

In [84]:
%load_ext dotenv
%dotenv -o

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [88]:
DB_HOST = os.getenv("DB_HOST")
DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")

In [90]:
# Establish database connection

graph = Graph(DB_HOST, auth=(DB_USER, DB_PASS))

In [91]:
matcher = NodeMatcher(graph)

In [31]:
#Open documents url for scraping
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(DOCUMENTS_URL)

content = driver.page_source
soup = BeautifulSoup(content)

driver.close()

[WDM] - Current google-chrome version is 83.0.4103
[WDM] - Get LATEST driver version for 83.0.4103
[WDM] - Driver [C:\Users\Joel\.wdm\drivers\chromedriver\win32\83.0.4103.39\chromedriver.exe] found in cache
 


In [32]:
"""Creating Immigration Document Nodes"""
docs = soup.find(attrs={'class': 'section-list'})
for doc_list_item in docs.findAll('li'):
    title = doc_list_item.a.find('span', attrs={
        'class': 'subsection-title-text'}
        ).text
    summary = doc_list_item.a.find('span', attrs={
        'class': 'subsection-summary'}
        ).text
    url = GOV_UK_HOME_URL + doc_list_item.a['href']
    
    tx = graph.begin()
    tx.evaluate('''
        CREATE (doc:Document {
            title: $title, summary: $summary, url: $url
            })
        ''', parameters = {'title': title, 'summary':summary, 'url': url})
    tx.commit()

In [92]:
"""Creating Section nodes related to all document nodes"""

# Query graph for all document nodes
docs = list(matcher.match("Document").skip(1)) #Skip immigration rules index document

# Loop through docs
for doc in docs:
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(doc['url']) # Get url from document node, open page for scraping

    content = driver.page_source
    soup = BeautifulSoup(content)

    driver.close()

    # Create section nodes connected to each doc
    for section in soup.findAll(attrs={'class': 'js-subsection-title'}):
        section_url = doc['url'] + '#' + section['id']

        tx = graph.begin()
        tx.evaluate('''
            CREATE (sec:Section {title: $title, url: $url})
            ''', parameters = {'title': section.text, 'url': section_url})
        tx.evaluate('''
            MATCH (doc:Document), (sec:Section)
            WHERE sec.title = $title AND doc.title = $doc_title AND NOT ()-[: CONTAINS]->(sec)
            CREATE (doc)-[: CONTAINS]->(sec)
            ''', parameters = {'title': section.text, 'doc_title': doc['title']})
        tx.commit()

[WDM] - Current google-chrome version is 83.0.4103
[WDM] - Get LATEST driver version for 83.0.4103
[WDM] - Get LATEST driver version for 83.0.4103
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/83.0.4103.39/chromedriver_win32.zip
 
[WDM] - Driver has been saved in cache [C:\Users\Joel\.wdm\drivers\chromedriver\win32\83.0.4103.39]


KeyboardInterrupt: 

In [34]:
"""Adding Paragraph labels to Section nodes which are also Paragraphs"""
tx = graph.begin()
tx.evaluate('''
    MATCH (s:Section), (d:Document)
    WHERE s.title =~ "^[0-9].*" AND NOT d.title CONTAINS 'Appendix' AND (d)-[:CONTAINS]-(s)
    SET s:Paragraph
    ''')
tx.commit()

In [35]:
"""Adding index property to Sections which are Paragraphs"""
sec_pars = list(matcher.match("Section").where("_: Paragraph")) # match sections which are also paragraphs

for sec_par in sec_pars:
    title = sec_par['title']
    index = title.split('.')[0]

    tx = graph.begin()
    tx.evaluate('''
        MATCH (p:Paragraph)
        WHERE p:Section AND p.title = $title
        SET p.index = $index
        ''', parameters = {'title': title, 'index': index})
    tx.commit()

In [56]:
#Open documents url for scraping
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(POINT_BASED_SYSTEM_URL)

content = driver.page_source
soup = BeautifulSoup(content)

driver.close()

[WDM] - Current google-chrome version is 83.0.4103
[WDM] - Get LATEST driver version for 83.0.4103
[WDM] - Driver [C:\Users\Joel\.wdm\drivers\chromedriver\win32\83.0.4103.39\chromedriver.exe] found in cache
 


In [47]:
"""Creating Paragraph Nodes for Paragraphs under the Sections of the Point Based System Part 6a Document"""
for section in soup.findAll(attrs={'class': 'js-openable'}):
    for tag in section.div.contents: # Loop through tags in section
        if(tag.name == 'h3' and tag.text != 'Notes'): # If it is an h3 tag and the tag's text is not Notes
            par_title_parts = tag.text.split('.')
            title = par_title_parts[-1].strip()
            index = par_title_parts[0]

            tx = graph.begin()
            tx.evaluate('''
                CREATE (par:Paragraph {title: $title, index: $index})
                ''', parameters = {'title': title, 'index': index})
            tx.evaluate('''
                MATCH (sec:Section), (par:Paragraph), (doc: Document)
                WHERE sec.title = $sec_title AND par.index = $index AND doc.title CONTAINS 'Appendix C:' AND NOT (doc)-[:CONTAINS]->(sec)
                CREATE (sec)-[: CONTAINS]->(par) 
                ''', parameters = {'sec_title': section.h2.text, 'index': index})
            tx.commit()


In [53]:
len(soup)

178

In [67]:
"""Extracting the contents/texts of each paragraph into a list for further processing"""
for section in soup.findAll(attrs={'class': 'js-openable'}):
    if section.h2.text == 'Tier 4 (General) Student':
        section_tags = section.div.findAll({'p', 'li', 'h3'})
        break


In [77]:
"""Adding rules to paragraphs"""
for tag in section_tags:
    if tag.name == 'h3' and tag.text != 'Notes':
        par_index = tag.text.split('.')[0]

    elif tag.name != 'h3':
        rule = tag.text
        
        tx = graph.begin()
        tx.evaluate('''
            CREATE (rule:Rule {desc: $desc})
            ''', parameters = {'desc': rule})
        tx.evaluate('''
            MATCH (par:Paragraph), (rule:Rule)
            WHERE par.index = $index and rule.desc = $desc AND NOT ()-[:CONTAINS]->(rule)
            CREATE (par)-[:CONTAINS]->(rule)
            ''', parameters = {'index': par_index, 'desc': rule})
        tx.commit()

In [50]:
paragraphs_soup = []
for paragraph in paragraphs:
    content = BeautifulSoup(paragraph)
    paragraphs_soup.append(content.findAll({'h3','li', 'p'}))

for soup in paragraphs_soup:
    rules = []
    for i, tag in enumerate(soup):
        print(i)
        if(tag.name == 'h3' and tag.text != 'Notes'):
            par_index = tag.text.split('.')[0]
            print(par_index)
            continue
        if(tag.name == 'p' and i < 3): #If p tag follows directly after h3 tag it is the desc of a par
            par_desc = tag.text
            print(par_desc)
            continue
        if(tag.name == 'li' and tag.name == 'p' and i > 2):
            rules.append(tag.text)
            continue

    # tx = graph.begin()
    # tx.evaluate('''
    #     MATCH (par:Paragraph)
    #     WHERE par.index = $index
    #     SET par.desc = $desc
    #     ''', parameters = {'index': par_index, 'desc': par_desc})
    # tx.commit()

    # for rule in rules:
    #     print(rule)

0
245ZT
1
This route is for migrants aged 16 or over who wish to study in the UK at an institution that is not a state funded school – except for voluntary grammar schools with boarding in Northern Ireland – and academies.
0
245ZT
1
This route is for migrants aged 16 or over who wish to study in the UK at an institution that is not a state funded school – except for voluntary grammar schools with boarding in Northern Ireland – and academies.
2
245ZU
3
0
245ZT
1
This route is for migrants aged 16 or over who wish to study in the UK at an institution that is not a state funded school – except for voluntary grammar schools with boarding in Northern Ireland – and academies.
2
245ZU
3
4
245ZV
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
0
245ZT
1
This route is for migrants aged 16 or over who wish to study in the UK at an institution that is not a state funded school – except for voluntary gramm